In [1]:
### Problems and Solutions

In [2]:
## 198 Problem 1 - Get total Aggregations

In [1]:
from pyspark.sql import SparkSession
import getpass

username = getpass.getuser()

spark = SparkSession. \
        builder. \
        config('spark.ui.port','0'). \
        config('spark.sql.warehouse.dir', f'/user/{username}/warehouse/'). \
        config('spark.shuffle.io.connectionTimeout','6000'). \
        config("spark.driver.memory", '6g'). \
        config('spark.executor.memory', '6g'). \
        enableHiveSupport(). \
        appName(f'{username} | Python - Basic Transformations'). \
        master('yarn'). \
        getOrCreate()

#        config('spark.executor.heartbeatInterval','20s'). \
#         config('spark.network.timeout','6000'). \

In [2]:
airtraffic_path = '/public/airlines_all/airlines-part/flightmonth=200801'

In [3]:
airtraffic = spark.read.parquet(airtraffic_path)

In [4]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [5]:
airtraffic.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

In [6]:
airtraffic.filter("IsArrDelayed='YES' AND Cancelled=0").show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

In [7]:
airtraffic.filter((airtraffic['IsArrDelayed']=='YES') & (airtraffic['Cancelled']==0)).show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

In [8]:
airtraffic.filter((airtraffic.IsArrDelayed=='YES') & (airtraffic.Cancelled==0)).show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

In [9]:
from pyspark.sql.functions import col, lit, count,sum, expr

In [10]:
airtraffic. \
    filter('Cancelled = 0'). \
    agg(
        count(lit(1)).alias('FlightCount'),
        sum(expr("CASE WHEN IsDepDelayed='YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed='YES' THEN 1 ELSE 0 END")).alias("ARRDelayedCount")
    ). \
    show()

+-----------+---------------+---------------+
|FlightCount|DepDelayedCount|ARRDelayedCount|
+-----------+---------------+---------------+
|     588366|         247905|         280663|
+-----------+---------------+---------------+



In [11]:
247905+280663

528568

In [12]:
from pyspark.sql.functions import when

In [13]:
airtraffic. \
    filter('Cancelled = 0'). \
    agg(
        count(lit(1)).alias('FlightCount'),
        sum(when(col('IsDepDelayed') == lit('YES'),1).otherwise(0)).alias("DepDelayedCount"),
        sum(when(col('IsArrDelayed') == lit('YES'),1).otherwise(0)).alias("ARRDelayedCount")
    ). \
    show()

+-----------+---------------+---------------+
|FlightCount|DepDelayedCount|ARRDelayedCount|
+-----------+---------------+---------------+
|     588366|         247905|         280663|
+-----------+---------------+---------------+



In [14]:
airtraffic. \
    filter('Cancelled = 0'). \
    agg(
        count(lit(1)).alias('FlightCount'),
        sum(when(col('IsDepDelayed') == 'YES',1).otherwise(0)).alias("DepDelayedCount"),
        sum(when(col('IsArrDelayed') == 'YES',1).otherwise(0)).alias("ARRDelayedCount")
    ). \
    show()

+-----------+---------------+---------------+
|FlightCount|DepDelayedCount|ARRDelayedCount|
+-----------+---------------+---------------+
|     588366|         247905|         280663|
+-----------+---------------+---------------+



In [15]:
from pyspark.sql.functions import concat,lpad,col,lit,expr

In [16]:
## 199 Solution Problem 2 Get Total Agregations by FlightDate


airtraffic.\
    filter('Cancelled=0'). \
    groupBy(
        concat(
            'Year',lit('-'),
            lpad(col('Month'),2,'0'),lit('-'),
            lpad(col('DayOfMonth'),2,'0')
        ).alias("FlightDate")
    ). \
    agg(
        count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed='YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed='YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    show()
    

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-15|      19204|           5304|           6389|
|2008-01-21|      19658|          10055|          11032|
|2008-01-11|      19825|           7255|           8164|
|2008-01-19|      15373|           6399|           6810|
|2008-01-02|      20442|          13294|          13749|
|2008-01-06|      19210|          10542|          10705|
|2008-01-29|      18596|           6324|           8370|
|2008-01-30|      19072|           6655|           7814|
|2008-01-17|      19401|           9635|          11229|
|2008-01-31|      19179|           9127|          11304|
|2008-01-01|      18623|          10501|          11173|
|2008-01-24|      19935|           8112|           9972|
|2008-01-07|      19762|           8122|           8683|
|2008-01-09|      19443|           5962|           6857|
|2008-01-04|      20160|       

In [22]:
airtraffic.\
    filter('Cancelled=0'). \
    groupBy(
        concat(
            'Year',lit('-'),
            lpad(col('Month'),2,'0'),lit('-'),
            lpad(col('DayOfMonth'),2,'0')
        ).alias("FlightDate")
    ). \
    agg(
        count(lit(1)).alias("FlightCount"),
        sum(when(col('IsDepDelayed')=='YES',1).otherwise(0)).alias("DepDelayedCount"),
        sum(when(col('IsArrDelayed')=='YES',1).otherwise(0)).alias("ArrDelayedCount")
       ). \
    show()

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-15|      19204|           5304|           6389|
|2008-01-21|      19658|          10055|          11032|
|2008-01-11|      19825|           7255|           8164|
|2008-01-19|      15373|           6399|           6810|
|2008-01-02|      20442|          13294|          13749|
|2008-01-06|      19210|          10542|          10705|
|2008-01-29|      18596|           6324|           8370|
|2008-01-30|      19072|           6655|           7814|
|2008-01-17|      19401|           9635|          11229|
|2008-01-31|      19179|           9127|          11304|
|2008-01-01|      18623|          10501|          11173|
|2008-01-24|      19935|           8112|           9972|
|2008-01-07|      19762|           8122|           8683|
|2008-01-09|      19443|           5962|           6857|
|2008-01-04|      20160|       

In [23]:
airtraffic.\
    filter('Cancelled=0'). \
    groupBy(
        concat(
            'Year',lit('-'),
            lpad(col('Month'),2,'0'),lit('-'),
            lpad(col('DayOfMonth'),2,'0')
        ).alias("FlightDate")
    ). \
    agg(
        count(lit(1)).alias("FlightCount"),
        sum(when(col('IsDepDelayed')=='YES',1).otherwise(0)).alias("DepDelayedCount"),
        sum(when(col('IsArrDelayed')=='YES',1).otherwise(0)).alias("ArrDelayedCount")
       ). \
    orderBy('FlightDate'). \
    show()

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-01|      18623|          10501|          11173|
|2008-01-02|      20442|          13294|          13749|
|2008-01-03|      20462|          11819|          12013|
|2008-01-04|      20160|           9406|           9824|
|2008-01-05|      17610|           9051|           9345|
|2008-01-06|      19210|          10542|          10705|
|2008-01-07|      19762|           8122|           8683|
|2008-01-08|      19140|           7483|           8938|
|2008-01-09|      19443|           5962|           6857|
|2008-01-10|      19956|           7033|           8565|
|2008-01-11|      19825|           7255|           8164|
|2008-01-12|      16346|           3902|           4078|
|2008-01-13|      18587|           6634|           7473|
|2008-01-14|      19267|           5921|           7104|
|2008-01-15|      19204|       

In [24]:
airtraffic.\
    filter('Cancelled=0'). \
    groupBy(
        concat(
            'Year',lit('-'),
            lpad(col('Month'),2,'0'),lit('-'),
            lpad(col('DayOfMonth'),2,'0')
        ).alias("FlightDate")
    ). \
    agg(
        count(lit(1)).alias("FlightCount"),
        sum(when(col('IsDepDelayed')=='YES',1).otherwise(0)).alias("DepDelayedCount"),
        sum(when(col('IsArrDelayed')=='YES',1).otherwise(0)).alias("ArrDelayedCount")
       ). \
    orderBy(col('FlightDate').desc()). \
    show()

+----------+-----------+---------------+---------------+
|FlightDate|FlightCount|DepDelayedCount|ArrDelayedCount|
+----------+-----------+---------------+---------------+
|2008-01-31|      19179|           9127|          11304|
|2008-01-30|      19072|           6655|           7814|
|2008-01-29|      18596|           6324|           8370|
|2008-01-28|      19493|           7580|           9013|
|2008-01-27|      18265|           8905|          10331|
|2008-01-26|      15860|           5740|           7163|
|2008-01-25|      19787|           8826|          10479|
|2008-01-24|      19935|           8112|           9972|
|2008-01-23|      19239|           7349|           9324|
|2008-01-22|      18716|           9129|          10669|
|2008-01-21|      19658|          10055|          11032|
|2008-01-20|      18406|           6700|           7005|
|2008-01-19|      15373|           6399|           6810|
|2008-01-18|      20117|          10038|          10860|
|2008-01-17|      19401|       